In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji
import re

## Entrenamiento de modelos supervisados para sentiments


Esta primera sección del notebook corresponde con el método de cargar df, se usa para entrenar los diferentes modelos implícitamente ya que siempre que se entrena un modelo de identificacion de sentimientos se usa el mismo df, por lo que es indispensable para el proceso de entrenamiento, el método lo que hace es cargar de los diferentes archivos .csv reunidos por todos los grupos los datos tageados, los normaliza y dependiendo de los parámetros de entrada del método también los lematizar y los balancea, además de normalizar las etiquetas de sentimientos como 0 para sentimientos positivos, 1 para sentimientos negativos y 2 para sentimientos neutrales, en la parte de lematización es importante resaltar que se usan lematizadores diferentes para ingles y español.

#### Carga de datos:
Para la carga de datos el método lee la carpeta de datos para sentimientos, donde se encuentran enumerados los archivos .csv, dependiendo del número de archivo el método trata al archivo de una manera u otra, ya que los rangos de números corresponden con los diferentes equipos, siendo los datos de 1 de aerolineas, 2, 3 y 4 de un equipo, 5, 6 y 7 de otro y 8 del ultimo, por esto se tratan de formas distintas, en esta primera parte se cargan solo las columnas de texto, idioma y el tag de sentimientos, por lo que además se deben renombrar algunas columnas en ciertos archivos, luego de esto se eliminan las filas que tengan valores nulos en cualquier columna, se eliminan las duplicados usando la columna texto com indicador y se filtran los datos cargados de tal forma que solo pasen los datos que tienen como idiomas ingles o español y como tag 0, 1 o 2, además se reinician los índices del df, posterior a esto usando expresiones regulares se eliminan los saltos de línea los espaciados de más de un espacio, los url, y los espacios al principio y fin de cada texto en el df, esto para normalizar los datos entregados.

#### Lematizacion:
La lematización, que es indicada por la segunda variable de entrada del método, la cual es binaria, se vale de la columna de idioma o lang para lematizar el texto usando un lematizador para ingles o español, para esto recorre todo el df identificando el idioma con la columna lang y lematizando el texto uno a uno según el lematizador correspondiente.

#### Balanceamiento de datos:
Por último para el balanceamiento de los datos se usa la primera variable del método, la cual también es binaria, para balancear el df de salida, para esto se crea una columna adicional en el df llamada sello, en sello se usan los atributos de idioma y de mourning en conjunto para crear una marca en el df según estos dos atributos, luego de completar los sellos se usan estos para que todos los sellos queden con la misma cantidad de datos en el df usando un muestreo aleatorio en df que fueron agrupados y separados del resto según su sello o marca, la cantidad de datos en cada muestra se determina usando la cantidad del df más pequeño resultante de la separación y agrupación por sellos, luego se unen de nuevo los df de muestras y se elimina la columna sellos, de esta forma la cantidad de datos de idioma y mourning se encuentran iguales en el df de salida en cada posible combinación.


In [2]:
def get_feelings_df(balance_data, lematizacion):
    from gensim.utils import any2unicode as unicode
    from nltk.stem import SnowballStemmer
    from sklearn.utils import resample
    import os, re, sys, pandas, unidecode

    feelings_folder = './entrenamiento de modelos/datos sentimientos'
    feelings_df = pandas.DataFrame(columns=['text', 'lang', 'sentiment'])
    path, subfolders, files_list = list(os.walk(feelings_folder))[0]
    files_list.sort()

    for i in range(len(files_list)):
        sys.stdout.write("\rPreparando df  " + str(round(((i + 1) / (len(files_list))) * 100, 2)) + "%")
        sys.stdout.flush()
        file_name, file_ext = files_list[i].split(".")

        if file_ext == 'csv':
            file_path = path + "/" + file_name + "." + file_ext
            df = pandas.read_csv(file_path, encoding='utf8', dtype=str, engine='python')
            numero_de_archivo = int(file_name.split("_")[0])

            if numero_de_archivo == 1:
                df = df.filter(['airline_sentiment', 'text'])
                df.columns = ['sentiment', 'text']
                df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 2 or numero_de_archivo == 3 or numero_de_archivo == 4:
                df = df.filter(['sentiment', 'text'])
                df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 5 or numero_de_archivo == 6 or numero_de_archivo == 7:
                df = df.filter(['polarity', 'text'])
                df.columns = ['sentiment', 'text']
                if numero_de_archivo in [5, 6]: df['lang'] = 'es'
                if numero_de_archivo == 7: df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 8:
                df = df.filter(['sentiment', 'text'])
                df.columns = ['sentiment', 'text']
                df['lang'] = 'es'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

    del df
    print("")
    feelings_df.dropna()
    feelings_df.drop_duplicates(subset=['text'], inplace=True)
    feelings_df = feelings_df.loc[feelings_df['sentiment'].isin(['0', '1', '2'])]
    feelings_df = feelings_df.loc[feelings_df['lang'].isin(['es', 'en'])]
    feelings_df.reset_index(drop=True, inplace=True)

    for i, row in feelings_df.iterrows():
        sys.stdout.write("\rNormalizando df " + str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        feelings_df.at[i, 'text'] = (
            re.sub(' +', ' ', re.sub("http\S+", "", re.sub('\s+', ' ', str(feelings_df.at[i, 'text']))))
        ).strip()

    print("")

    if balance_data == 1:
        feelings_df.dropna()
        feelings_df["Sello"] = 0
        for i, row in feelings_df.iterrows():
            #sys.stdout.write(
             #   "\rCreando sellos de balanceamiento " +
             #   str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2))
             #   + "%"
            #)
            #sys.stdout.flush()
            feelings_df.at[i, 'sello'] = str(feelings_df.at[i, 'lang']) + '_' + str(feelings_df.at[i, 'sentiment'])
        print("\nBalanceando df")
        min_len1 = int(min(feelings_df['sello'].value_counts()))
        df_0 = resample(feelings_df[feelings_df.sello == 'en_0'], replace=False, n_samples=min_len1, random_state=1)
        df_1 = resample(feelings_df[feelings_df.sello == 'en_1'], replace=False, n_samples=min_len1, random_state=1)
        df_2 = resample(feelings_df[feelings_df.sello == 'en_2'], replace=False, n_samples=min_len1, random_state=1)
        df_3 = resample(feelings_df[feelings_df.sello == 'es_0'], replace=False, n_samples=min_len1, random_state=1)
        df_4 = resample(feelings_df[feelings_df.sello == 'es_1'], replace=False, n_samples=min_len1, random_state=1)
        df_5 = resample(feelings_df[feelings_df.sello == 'es_2'], replace=False, n_samples=min_len1, random_state=1)
        feelings_df = pandas.concat([df_0, df_1, df_2, df_3, df_4, df_5])
        feelings_df = feelings_df.filter(['text', 'lang', 'sentiment'])
        feelings_df.reset_index(drop=True, inplace=True)

    feelings_df.reset_index(drop=True, inplace=True)

    if lematizacion == 1:
        stemmer_en = SnowballStemmer('english')
        stemmer_es = SnowballStemmer('spanish')
        for i, row in feelings_df.iterrows():
            sys.stdout.write("\rLematizando df " + str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2)) + "%")
            sys.stdout.flush()
            if type(feelings_df.at[i, 'text']) is str and feelings_df.at[i, 'lang'] == 'es':
                feelings_df.at[i, 'text'] = str(stemmer_es.stem(feelings_df.at[i, 'text'])).lower()
            elif type(feelings_df.at[i, 'text']) is str and feelings_df.at[i, 'lang'] == 'en':
                feelings_df.at[i, 'text'] = str(stemmer_en.stem(feelings_df.at[i, 'text'])).lower()
        print("")

    feelings_df.reset_index(drop=True, inplace=True)

    print("Df sentiments entregado\n")
    feelings_df.reset_index(drop=True, inplace=True)
    
    data12 = pd.DataFrame(columns=('text', 'lang', 'sentiment','emoji'))
    emoj=0
    tweet_tokenizer = TweetTokenizer()

    for row in feelings_df.iterrows():
        texto = row[1]['text']
        text = re.sub(r'[\b@]\w+\s{1}', '', texto)#Quitar menciones
        words = tweet_tokenizer.tokenize(text)
        for w in words:
            if  w in emoji.UNICODE_EMOJI:
                emoj = 1
                break
            else: emoj=0

        lenguaje = row[1]['lang']
        sentiment = row[1]['sentiment']
        data12.loc[len(data12)]=[text,lenguaje,sentiment,emoj] 
        feelings_df = data12
    
    return feelings_df

In [3]:
df = get_feelings_df(1,1)

Normalizando df 13.15%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Normalizando df 27.26%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Normalizando df 42.99%

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Lematizando df 75.78%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Lematizando df 99.24%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
print("\nTweets en el dataframe original: " + str(df.shape[0]))
df.columns.values
df1 = df


Tweets en el dataframe original: 15564


In [7]:
#----------------- Lexicones
#Lectura de SentiWordNet Obtenido de 
#https://www.nltk.org/_modules/nltk/corpus/reader/sentiwordnet.html

# SentiWordNet[word] = {POS,	ID,	PosScore,	NegScore}
contador = 0
SentiWordNet = dict()
for lines in open('./entrenamiento de modelos/data lexicon/SentiWordNet_3.0.0.txt'):
    if lines.startswith('#'):
        continue
    line = lines.split('\t')
    palabra = line[4].split('#')[0]
    if (palabra in SentiWordNet) or (palabra==''):
        continue
    else:
        SentiWordNet[palabra]={'POS': line[0], 'ID': line[1], 'PosScore': line[2], 'NegScore': line[3]}

In [8]:
# AFFIN[word] = sentiment
AFFIN = dict()
for lines in open('./entrenamiento de modelos/data lexicon/AFFIN-111.txt'):
    AFFIN[lines.split('\t')[0]]=(lines.split('\t')[1]).split('\n')[0]

In [9]:
# Lexicon Esp Pos - Neg[word] = sentiment
ESPpos = list()
ESPneg = list()
for lines in open('./entrenamiento de modelos/data lexicon/negative_words_es.txt'):
    ESPneg.append(lines.split("\n")[0])
for lines1 in open('./entrenamiento de modelos/data lexicon/positive_words_es.txt'):
    ESPpos.append(lines1.split("\n")[0])

In [10]:
tt = TweetTokenizer()
def Lexicon(data1):
    data = data1['text']
    addlex = list()
    i=0
    for frase in data:
        splited = tt.tokenize(frase)
        
        sum_swn_neg = 0
        sum_swn_pos = 0
        affin = 0
        stnet = 0
        contpos = 0
        contneg = 0
        word_stat = 0
        total=0
        for word in splited:
        
            #if word in SentiWordNet.keys():
            #    sum_swn_neg += float(SentiWordNet[word]['NegScore'])
            #    sum_swn_pos += float(SentiWordNet[word]['PosScore'])
            #if word in AFFIN.keys():
            #    affin += float(AFFIN[word])
            if word in ESPpos:
                contpos +=1
            if word in ESPneg:
                contneg +=1
        total = contpos + contneg
        if total!=0:
            total = contpos / total
        else: total=0.5
        #datemo = int(data1['emoji'][i])
        addlex.append([total])
        i+=1
    return addlex

In [11]:
tt = TweetTokenizer()
def Lexicon1(data1):
    data = data1['text']
    addlex = list()
    i=0
    for frase in data:
        splited = tt.tokenize(frase)
        
        sum_swn_neg = 0
        sum_swn_pos = 0
        affin = 0
        stnet = 0
        contpos = 0
        contneg = 0
        word_stat = 0
        total=0
        for word in splited:
        
            if word in SentiWordNet.keys():
                sum_swn_neg += float(SentiWordNet[word]['NegScore'])
                sum_swn_pos += float(SentiWordNet[word]['PosScore'])
            if word in AFFIN.keys():
                affin += float(AFFIN[word])
            #if word in ESPpos:
            #    contpos +=1
            #if word in ESPneg:
            #    contneg +=1
        #total = contpos + contneg
        #if total!=0:
            #total = contpos / total
        #else: total=0.5
        datemo = int(data1['emoji'][i])
        addlex.append([sum_swn_neg,sum_swn_pos,affin,datemo])
        i+=1
    return addlex

In [12]:
def extension(matriz, data):
    extendLex = np.array(Lexicon(data))
    return np.append(matriz, extendLex, 1)

def extensionen(matriz, data):
    extendLex = np.array(Lexicon1(data))
    return np.append(matriz, extendLex, 1)

In [13]:
vectorizer = TfidfVectorizer(max_features=3000, min_df=7, max_df=0.8)

# -------- Español.
df_es = df[df1.lang == 'es']
# -------- Ingles.
df_en = df[df1.lang == 'en']

In [14]:
df_es.shape
df_en.shape

(7782, 4)

In [15]:
features = vectorizer.fit_transform(df_es['text']).toarray()
featuresen = vectorizer.fit_transform(df_en['text']).toarray()

labels = df_es['sentiment']
labelsen = df_en['sentiment']

In [16]:
labels.shape
labelsen.shape

(7782,)

In [17]:
vectorizacionLexicon = extension(features, df_es)
vectorizacionLexicon.shape

vectorizacionLexiconen = extensionen(featuresen, df_en)
vectorizacionLexiconen.shape

(7782, 1696)

#### En esta segunda sección del notebook está el código necesario para entrenar los diferentes modelos supervisados de mourning dispuesto como método para ser implementado y usado fácilmente, este método se vale del de get_mourning_df, ya que este provee el df necesario para entrenar los diferentes modelos supervisados que se quieren entrenar.

El pipeline seguido para entrenar los modelos de mourning se basa primero en seleccionar el modelo que se va a entrenar, este se selecciona con el primer parámetro del método, el cual es una cadena donde se escriben las iniciales del modelo: 

GBT: para gradient boosting trees.
NN: para MPL o redes neuronales.
DT: para árboles de decisión.
RF: para random forest.
NB: para naive bayes.

Luego de seleccionar el modelo se carga el df haciendo uso del método get_mourning_df, los parámetros de este corresponden con los dos restantes del método de entrenamiento, luego de cargar el df este se divide según su idioma en 2 df, esto debido a que se entrenan modelos separados para ingles y español para integrar correctamente los lexicones posteriormente según el idioma, luego de esto se separan los df en los datos y etiquetas de prueba, este procedimiento también es independiente para cada idioma, posteriormente se vectorizan los datos tambien segun su idioma y se entrenan y prueban los modelos, en esta etapa se guardan los vocabularios de cada vectorizador aparte y también los modelos, esto con el fin de hacer uso de estos en otro programa posteriormente.

Por último se muestran los resultados del modelo entrenado.


In [18]:
def entrenar_modelos_sentiments_supervisados(modelo_entr,vectorizacionLexicon,labels,vectorizacionLexicon1,labels1):
    from nltk.corpus import stopwords
    from sklearn.naive_bayes import GaussianNB,BernoulliNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import classification_report
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    import pandas as pd
    import pickle, os, nltk

    modelos_es = {
        'GBT': AdaBoostClassifier(DecisionTreeClassifier(max_depth=200), n_estimators=10),
        'RF': RandomForestClassifier(n_estimators=100, max_depth=300),
        'NN': MLPClassifier(hidden_layer_sizes=(50, 2), max_iter=100),
        'DT': DecisionTreeClassifier(max_depth=16),
        'NB': BernoulliNB()
    }
    
    modelos_en = {
        'GBT': AdaBoostClassifier(DecisionTreeClassifier(max_depth=200), n_estimators=10),
        'RF': RandomForestClassifier(n_estimators=100, max_depth=300),
        'NN': MLPClassifier(hidden_layer_sizes=(50, 2), max_iter=100),
        'DT': DecisionTreeClassifier(max_depth=16),
        'NB': BernoulliNB()
        
    }


    print("")
    nltk.download('stopwords')

    # ---------------- Asignacion de los modelos y vectorizadores.
    # -------- Español.
    modelo_es = modelos_es[modelo_entr]

    # -------- Ingles.
    modelo_en = modelos_en[modelo_entr]
        
    
    print("Separacion de datos por idioma terminada")
    # ---------------- Separacion en data y labels de entrenamiento.
    # -------- Español.
    data_train_es, data_test_es, label_train_es, label_test_es = train_test_split(
        vectorizacionLexicon, labels, random_state=1
    )
    # -------- Ingles.
    data_train_en, data_test_en, label_train_en, label_test_en = train_test_split(
        vectorizacionLexicon1, labels1, random_state=1
    )
    
    # ---------------- entrenamiento y guardado de los modelos.
    # -------- Español.
    ruta_modelo_es = './entrenamiento de modelos/modelos/' + str(type(modelo_es).__name__) + '_Sentiment_es.sav'
    modelo_es.fit(data_train_es, label_train_es)
    pickle.dump(modelo_es, open(ruta_modelo_es, 'wb'))
    print("Modelo de " + str(type(modelo_es).__name__) + " en español guardado en " + ruta_modelo_es)

    # -------- Ingles.
    ruta_modelo_en = './entrenamiento de modelos/modelos/' + str(type(modelo_en).__name__) + '_Sentiment_en.sav'
    modelo_en.fit(data_train_en, label_train_en)
    pickle.dump(modelo_en, open(ruta_modelo_en, 'wb'))
    print("Modelo de " + str(type(modelo_en).__name__) + " en ingles guardado en " + ruta_modelo_en)

    # ---------------- implementacion de los modelos.
    # -------- Español.
    predictions_es = modelo_es.predict(data_test_es)
    # -------- Ingles.
    predictions_en = modelo_en.predict(data_test_en)
    print("Predicciones terminadas")
    
    
    # ---------------- Resultados de los modelos.
    # -------- Español.
    print("\nResultados " + str(type(modelo_es).__name__) + " Español:\n")
    print(classification_report(label_test_es, predictions_es))
    # -------- Ingles.
    print("\nResultados " + str(type(modelo_en).__name__) + " Ingles:\n")
    print(classification_report(label_test_en, predictions_en))


#### Entrenamiento de modelos NB:

In [19]:
entrenar_modelos_sentiments_supervisados('NB',vectorizacionLexicon,labels,vectorizacionLexiconen,labelsen)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Separacion de datos por idioma terminada
Modelo de BernoulliNB en español guardado en ./entrenamiento de modelos/modelos/BernoulliNB_Sentiment_es.sav
Modelo de BernoulliNB en ingles guardado en ./entrenamiento de modelos/modelos/BernoulliNB_Sentiment_en.sav
Predicciones terminadas

Resultados BernoulliNB Español:

              precision    recall  f1-score   support

           0       0.88      0.82      0.85       634
           1       0.86      0.91      0.88       675
           2       0.77      0.76      0.77       637

    accuracy                           0.84      1946
   macro avg       0.83      0.83      0.83      1946
weighted avg       0.84      0.84      0.83      1946


Resultados BernoulliNB Ingles:

              precision    recall  f1-score   support

           0       0.77      0.74      0.75       634
           1       0.79      0.78      0.78       675
           2       0.69      0.73      0.71       637

    accuracy                           0.75      194

#### Entrenamiento de modelos DT:

In [20]:
entrenar_modelos_sentiments_supervisados('DT',vectorizacionLexicon,labels,vectorizacionLexiconen,labelsen)


Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de DecisionTreeClassifier en español guardado en ./entrenamiento de modelos/modelos/DecisionTreeClassifier_Sentiment_es.sav
Modelo de DecisionTreeClassifier en ingles guardado en ./entrenamiento de modelos/modelos/DecisionTreeClassifier_Sentiment_en.sav
Predicciones terminadas

Resultados DecisionTreeClassifier Español:

              precision    recall  f1-score   support

           0       0.96      0.75      0.84       634
           1       0.99      0.92      0.96       675
           2       0.75      0.97      0.84       637

    accuracy                           0.88      1946
   macro avg       0.90      0.88      0.88      1946
weighted avg       0.90      0.88      0.88      1946


Resultados DecisionTreeClassifier Ingles:

              precision    recall  f1-score   support

           0       0.70      0.62      0.66       634
           1       0.69      0.69      0.69       675
           2       0.58      0.65      0.61       637

    accuracy               

#### Entrenamiento de modelos RF:

In [21]:
entrenar_modelos_sentiments_supervisados('RF',vectorizacionLexicon,labels,vectorizacionLexiconen,labelsen)



Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de RandomForestClassifier en español guardado en ./entrenamiento de modelos/modelos/RandomForestClassifier_Sentiment_es.sav
Modelo de RandomForestClassifier en ingles guardado en ./entrenamiento de modelos/modelos/RandomForestClassifier_Sentiment_en.sav
Predicciones terminadas

Resultados RandomForestClassifier Español:

              precision    recall  f1-score   support

           0       0.91      0.82      0.87       634
           1       0.94      0.94      0.94       675
           2       0.79      0.87      0.83       637

    accuracy                           0.88      1946
   macro avg       0.88      0.88      0.88      1946
weighted avg       0.88      0.88      0.88      1946


Resultados RandomForestClassifier Ingles:

              precision    recall  f1-score   support

           0       0.76      0.72      0.74       634
           1       0.77      0.80      0.79       675
           2       0.70      0.70      0.70       637

    accuracy               

#### Entrenamiento de modelos GBT:


In [22]:
entrenar_modelos_sentiments_supervisados('GBT',vectorizacionLexicon,labels,vectorizacionLexiconen,labelsen)



Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Modelo de AdaBoostClassifier en español guardado en ./entrenamiento de modelos/modelos/AdaBoostClassifier_Sentiment_es.sav
Modelo de AdaBoostClassifier en ingles guardado en ./entrenamiento de modelos/modelos/AdaBoostClassifier_Sentiment_en.sav
Predicciones terminadas

Resultados AdaBoostClassifier Español:

              precision    recall  f1-score   support

           0       0.90      0.80      0.85       634
           1       0.98      0.93      0.95       675
           2       0.78      0.91      0.84       637

    accuracy                           0.88      1946
   macro avg       0.89      0.88      0.88      1946
weighted avg       0.89      0.88      0.88      1946


Resultados AdaBoostClassifier Ingles:

              precision    recall  f1-score   support

           0       0.67      0.67      0.67       634
           1       0.72      0.69      0.70       675
           2       0.58      0.61      0.60       637

    accuracy                           0.66      19

#### Entrenamiento de modelos NN:

In [23]:
entrenar_modelos_sentiments_supervisados('NN',vectorizacionLexicon,labels,vectorizacionLexiconen,labelsen)



Separacion de datos por idioma terminada


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipedev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/home/felipedev/Documentos/entornopruebas/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Modelo de MLPClassifier en español guardado en ./entrenamiento de modelos/modelos/MLPClassifier_Sentiment_es.sav
Modelo de MLPClassifier en ingles guardado en ./entrenamiento de modelos/modelos/MLPClassifier_Sentiment_en.sav
Predicciones terminadas

Resultados MLPClassifier Español:

              precision    recall  f1-score   support

           0       0.79      0.85      0.82       634
           1       0.83      0.88      0.85       675
           2       0.79      0.67      0.72       637

    accuracy                           0.80      1946
   macro avg       0.80      0.80      0.80      1946
weighted avg       0.80      0.80      0.80      1946


Resultados MLPClassifier Ingles:

              precision    recall  f1-score   support

           0       0.70      0.70      0.70       634
           1       0.75      0.70      0.72       675
           2       0.62      0.66      0.64       637

    accuracy                           0.69      1946
   macro avg       0.69    

/home/felipedev/Documentos/entornopruebas/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
